In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.0 MB/s eta 0:00:00


In [ ]:
!pip install accelerate>=0.20.1

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [ ]:
dataset_path = '/content/fine_tuning.csv'
output_dir = 'fine_tuned_gpt2_model'

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(dataset_path)
prompts = df['Device'].tolist()  # Convert to a list of strings
targets = df['Result'].tolist()

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [ ]:
# Tokenize the prompts and targets
tokenized_prompts = tokenizer(prompts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
tokenized_targets = tokenizer(targets, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

In [ ]:
from torch.utils.data import Dataset, DataLoader, TensorDataset

class CustomDataset(Dataset):
    def __init__(self, prompts, targets):
        self.prompts = prompts
        self.targets = targets

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):
        return {
            'input_ids': self.prompts[idx].input_ids,
            'attention_mask': self.prompts[idx].attention_mask,
            'labels': self.targets[idx].input_ids,
        }

train_dataset = CustomDataset(tokenized_prompts, tokenized_targets)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True)

tokenized_prompts = tokenize_function(prompts)

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=3,  # Adjust as needed
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_prompts,
)

In [ ]:
trainer.train()
trainer.save_model()

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned GPT-2 model
output_dir = 'fine_tuned_gpt2_model'  # Update this with your fine-tuned model's directory
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)

# Define a function to generate results based on device name
def generate_result(device_name):
    # Construct a prompt with the device name
    prompt = f"{device_name} API Description:"

    # Generate text using the model
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract the result from the generated text
    result_start = generated_text.find("API Description:") + len("API Description:")
    result_end = generated_text.find("Utility:")
    result = generated_text[result_start:result_end].strip()

    return result

# Get user input for the device name
device_name = input("Enter the device name:iphone 6 ")

# Generate and print the result
result = generate_result(device_name)
print("Result:", result)



Device:
- iphone 6
api Description: - change the lock screen pattern of the phone.
utility:- security.
Device:
- iphone 6
api Description:- check battery health.
utility:- battery.
Device:
- iphone 6
api Description:- captures high resolution pictures using the camera.
utility:- camera.
Device:
- iphone 6
api Description:- using tablet mode.
utility:- functionality.



In [ ]:
!pip install transformers==4.10.0
!pip install accelerate>=0.20.1
!pip install torch==1.9.0
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
from torch.utils.data import Dataset



# Define the paths to your dataset and save directory
dataset_path = '/content/fine_tuning.csv'
output_dir = 'fine_tuned_gpt2_model'

# Load the dataset
import pandas as pd

df = pd.read_csv(dataset_path)
prompts = df['Device'].tolist()  # Convert to a list of strings
targets = df['Result'].tolist()  # Convert to a list of strings

# Initialize the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Add a padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize the prompts and targets
tokenized_prompts = tokenizer(prompts, padding=True, truncation=True, max_length=128, return_tensors="pt", add_special_tokens=True)
tokenized_targets = tokenizer(targets, padding=True, truncation=True, max_length=128, return_tensors="pt", add_special_tokens=True)

# Combine prompts and targets into a dataset
class CustomDataset(Dataset):
    def init(self, prompts, targets):
        self.prompts = prompts
        self.targets = targets

    def len(self):
        return len(self.prompts.input_ids)

    def getitem(self, idx):
        return {
            'input_ids': self.prompts.input_ids[idx],
            'attention_mask': self.prompts.attention_mask[idx],
            'labels': self.targets.input_ids[idx],
        }

train_dataset = CustomDataset(tokenized_prompts, tokenized_targets)

# Define training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=3,  # Adjust as needed
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=None,  # We're using a custom dataset and data collator
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model()

# Optionally, you can evaluate the model on a validation dataset and generate test cases using the fine-tuned model.